#### Define 'EPOCHS' (total epochs) and 'EP_INT' (interval epochs) and then run from the starting...

In [49]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import  vocab,data
from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator, TabularDataset
from torchvision.transforms import ToTensor

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os, csv, sys, random, re, time, math, spacy, nltk

from PIL import Image
from numpy.random import RandomState
from tensorboardX import SummaryWriter
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [50]:
nlp = spacy.load('en_core_web_sm')

#Define the logger
#log_writer_train = SummaryWriter('TBlogs/train/')
#log_writer_val = SummaryWriter('TBlogs/val/')
#log_writer_test = SummaryWriter('TBlogs/test/')

log_writer = SummaryWriter('TBlogs/')

In [51]:
EPOCHS = 3     # Total epochs to train for
EP_INT = 1     # In the intervals of 'EP_INT' epochs
CLIP = 1

NF=1000        # For ResNet/VGG Feature Extraction
#NF = 36*2048  # For Faster RCNN Feature Extraction using Object Detection

In [52]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [53]:
dataset=pd.read_csv(r"train_data.csv")

In [54]:
dataset.head(-1)

,src,trg,img_path,img_id
0,it was a dry year at the globes for films dire...,so welldeserved she was brilliant in this movi...,/home/puneet/code/Multimodal Feedback/data/139...,1398
1,a person was taken into custody in the killing...,probably knew the clinton cabal,/home/puneet/code/Multimodal Feedback/data/170...,1703
2,new jersey at very dangerous place over corona...,trump,/home/puneet/code/Multimodal Feedback/data/277...,2770
3,image copyright getty images image caption rut...,rbg you stay alive stay alive,/home/puneet/code/Multimodal Feedback/data/203...,2036
4,per person albertans donate more money to char...,this is a great example of how socialist and c...,/home/puneet/code/Multimodal Feedback/data/329...,329
...,...,...,...,...
995,image copyright getty images image caption the...,we look like we have snouts try a face shield ...,/home/puneet/code/Multimodal Feedback/data/317...,3177
996,dr anthony fauci the nations leading infectiou...,hard pass,/home/puneet/code/Multimodal Feedback/data/370...,3709
997,a florida sheriff has banned his employees and...,the employees should walk out due to an unsafe...,/home/puneet/code/Multimodal Feedback/data/402...,4027
998,a federal appeals court friday overturned the ...,great now us tax payers have to pay to keep hi...,/home/puneet/code/Multimodal Feedback/data/259...,2594


In [55]:
def tokenize_text(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in nlp.tokenizer(text)]

In [56]:
ID = data.Field(sequential=False,use_vocab=False)
SRC = Field(tokenize = tokenize_text, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True,
            include_lengths = True)
TRG = Field(tokenize = tokenize_text, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True,
            )
datafields=[('src', SRC), ('trg', TRG),('img_path',None),('img_id',ID)]
datafields

[('src', <torchtext.data.field.Field at 0x7f8bc8d2b310>),
 ('trg', <torchtext.data.field.Field at 0x7f8bc8d2b110>),
 ('img_path', None),
 ('img_id', <torchtext.data.field.Field at 0x7f8bc8d2b250>)]

In [57]:
'''df = pd.read_csv("data.csv")

rng = RandomState()
train_data = df.sample(frac=0.7, random_state=rng)
val_data = df.loc[~df.index.isin(train_data.index)]

#cols= ["src", "trg", "img_path", "img_id"]
train_data.to_csv('train_data.csv', index= False) #columns=cols
val_data.to_csv('val_data.csv', index= False) #columns=cols
'''

'df = pd.read_csv("data.csv")\n\nrng = RandomState()\ntrain_data = df.sample(frac=0.7, random_state=rng)\nval_data = df.loc[~df.index.isin(train_data.index)]\n\n#cols= ["src", "trg", "img_path", "img_id"]\ntrain_data.to_csv(\'train_data.csv\', index= False) #columns=cols\nval_data.to_csv(\'val_data.csv\', index= False) #columns=cols\n'

In [58]:
train_data, val_data = data.TabularDataset.splits(path=r"",train="train_data.csv", validation="val_data.csv", format='csv', skip_header=True, fields=datafields)

In [59]:
print(len(train_data))
print(len(val_data))

1001
100


In [60]:
#print(vars(train_data.examples[4]))

In [61]:
sys.maxsize

9223372036854775807

In [62]:
SRC.build_vocab(train_data, vectors="glove.6B.100d")
TRG.build_vocab(train_data, vectors="glove.6B.100d")
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=1)

In [63]:
print(f"Unique tokens in target (en) vocabulary: {len(SRC.vocab)}")

Unique tokens in target (en) vocabulary: 18500


In [64]:
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in target (en) vocabulary: 4025


In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [66]:
device

device(type='cuda')

In [67]:
BATCH_SIZE = 4

train_iterator, x_iterator = BucketIterator.splits(
    (train_data, train_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    sort_key = lambda x : len(x.src), 
    device = device)

valid_iterator, y_iterator = BucketIterator.splits(
    (val_data, val_data), 
    batch_size = BATCH_SIZE, 
    sort_within_batch = True,
    sort_key = lambda x : len(x.src),
    device = device)

#valid_iterator, y_iterator = BucketIterator.splits(
#    (val_data, val_data), 
#    batch_size = BATCH_SIZE, 
#    device = device)

In [68]:
print(len(train_iterator))
print(len(valid_iterator))

251
25


In [69]:
for i, batch in enumerate(valid_iterator):
  print(i,batch)

0 
[torchtext.data.batch.Batch of size 4]
	[.src]:('[torch.cuda.LongTensor of size 1202x4 (GPU 0)]', '[torch.cuda.LongTensor of size 4 (GPU 0)]')
	[.trg]:[torch.cuda.LongTensor of size 43x4 (GPU 0)]
	[.img_id]:[torch.cuda.LongTensor of size 4 (GPU 0)]
1 
[torchtext.data.batch.Batch of size 4]
	[.src]:('[torch.cuda.LongTensor of size 254x4 (GPU 0)]', '[torch.cuda.LongTensor of size 4 (GPU 0)]')
	[.trg]:[torch.cuda.LongTensor of size 55x4 (GPU 0)]
	[.img_id]:[torch.cuda.LongTensor of size 4 (GPU 0)]
2 
[torchtext.data.batch.Batch of size 4]
	[.src]:('[torch.cuda.LongTensor of size 705x4 (GPU 0)]', '[torch.cuda.LongTensor of size 4 (GPU 0)]')
	[.trg]:[torch.cuda.LongTensor of size 54x4 (GPU 0)]
	[.img_id]:[torch.cuda.LongTensor of size 4 (GPU 0)]
3 
[torchtext.data.batch.Batch of size 4]
	[.src]:('[torch.cuda.LongTensor of size 1030x4 (GPU 0)]', '[torch.cuda.LongTensor of size 4 (GPU 0)]')
	[.trg]:[torch.cuda.LongTensor of size 41x4 (GPU 0)]
	[.img_id]:[torch.cuda.LongTensor of size 4 (GP

In [70]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.vis_fc=nn.Linear(enc_hid_dim * 8, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src,visual_features, src_len):
        
        #src = [src len, batch size]
        #src_len = [src len]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
                
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len)
                
        packed_outputs, hidden = self.rnn(packed_embedded)
                                 
        #packed_outputs is a packed sequence containing all hidden states
        #hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 

        vis_features= torch.tanh(self.vis_fc(visual_features))
            
        #outputs is now a non-packed sequence, all hidden states obtained
        #  when the input is a pad token are all zeros
            
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        text_hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        hidden = torch.tanh(self.fc(torch.cat((text_hidden,vis_features), 1)))
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden ,vis_features

In [71]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs,vis_features, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        #mask = [batch size, src len]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat encoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        vis_features=vis_features.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        vis_energy=torch.tanh(self.attn(torch.cat((vis_features, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]
                
        energy = energy.permute(0, 2, 1)
        vis_energy = vis_energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)
        
        #v = [batch size, 1, dec hid dim]
            
        attention = torch.bmm(v, energy).squeeze(1)
        vis_attention = torch.bmm(v, vis_energy).squeeze(1)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        vis_attention = vis_attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim = 1),F.softmax(vis_attention, dim = 1)

In [72]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 4) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 4) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs,vis_features, mask):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        #mask = [batch size, src len]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a,a_vis = self.attention(hidden, encoder_outputs,vis_features, mask)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        a_vis = a_vis.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        vis_weighted = torch.bmm(a_vis, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        vis_weighted = vis_weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted,vis_weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 4) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        vis_weighted = vis_weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted,vis_weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1), a_vis.squeeze(1)

In [73]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src,visual_features, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden,vis_features = self.encoder(src,visual_features, src_len)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        mask = self.create_mask(src)

        #mask = [batch size, src len]
                
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, rand_1,rand_2 = self.decoder(input, hidden, encoder_outputs,vis_features, mask)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
            
        return outputs

In [74]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 100
DEC_EMB_DIM = 100
ENC_HID_DIM = 125
#ENC_HID_DIM = 512
DEC_HID_DIM = 125
#DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

In [75]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token] 
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

In [76]:
INPUT_DIM

18500

In [77]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(18500, 100)
    (rnn): GRU(100, 125, bidirectional=True)
    (fc): Linear(in_features=250, out_features=125, bias=True)
    (vis_fc): Linear(in_features=1000, out_features=125, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=375, out_features=125, bias=True)
    )
    (embedding): Embedding(4025, 100)
    (rnn): GRU(600, 125)
    (fc_out): Linear(in_features=725, out_features=4025, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [78]:
def count_parameters(model): 
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,821,150 trainable parameters


In [79]:
optimizer = optim.Adam(model.parameters())

In [80]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [81]:
def train(model, iterator, optimizer, criterion, clip):#, log_writer_train):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src,src_len = batch.src
        trg = batch.trg
        img_id=batch.img_id
        
        #Log with Tensorboard: Loss & PPL for train & val 
#        log_writer_train.add_text('Text', str(src), epoch+1)
#        log_writer_train.add_text('Comment', str(trg), epoch+1)
##       log_writer_train.add_image('Image', img_id, epoch+1)    
        
        
        
        x=img_id.cpu().numpy()
        y=len(x)
        #visual_features=torch.empty(y,4096).cuda()
        visual_features=torch.empty(y,NF).cuda()
        df=pd.read_csv(r"visual_features_resnet.csv")
        for i in range(y):
            q=df[str(x[i])].to_numpy()
            a=torch.from_numpy(q).unsqueeze(0)
            visual_features[i]=a
        
        optimizer.zero_grad()

        #print(visual_features.shape) 
        #print(type(visual_features))   
        output = model(src,visual_features, src_len, trg)
        
        #trg = [output_dim, batch_size]
        #output = [output_dim, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(output_dim - 1) * batch size]
        #output = [(output_dim - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [82]:
def evaluate(model, iterator, criterion):#, log_writer_val):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src,src_len = batch.src
            trg = batch.trg
            img_id=batch.img_id
            
            x=img_id.cpu().numpy()
            y=len(x)
            
            #visual_features=torch.empty(y,4096).cuda()
            visual_features=torch.empty(y,NF).cuda()
            df=pd.read_csv(r"visual_features_resnet.csv")
            for i in range(y):
                q=df[str(x[i])].to_numpy()
                a=torch.from_numpy(q).unsqueeze(0)
                visual_features[i]=a

            output = model(src,visual_features, src_len, trg,0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [83]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [84]:
def load_checkpt(model, optimizer, chpt_file):
    start_epoch = 0
    if (os.path.exists(chpt_file)):
        print("=> loading checkpoint '{}'".format(chpt_file))
        checkpoint = torch.load(chpt_file)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        start_epoch = checkpoint['epoch']
        
        print("=> loaded checkpoint '{}' (epoch {})".format(chpt_file, checkpoint['epoch']))
        
    else:
        print("=> Checkpoint NOT found '{}'".format(chpt_file))
    return model, optimizer, start_epoch

In [85]:
def translate_sentence(sentence, src_field, trg_field, img_id,path, model, device, max_len = 50):

    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('en_core_web_sm')
        #nlp = spacy.load('en') ##https://www.gitmemory.com/issue/OmkarPathak/pyresparser/46/777568505
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    src_len = torch.LongTensor([len(src_indexes)]).to(device)
    
    df=pd.read_csv(path)
    visual_features=torch.empty(1,NF).cuda()
#    visual_features=torch.empty(1,4096).cuda()
    q=df[str(img_id)].to_numpy()
    a=torch.from_numpy(q).unsqueeze(0)
    visual_features[0]=a

    with torch.no_grad():
        encoder_outputs, hidden, vis_features = model.encoder(src_tensor,visual_features, src_len)

    mask = model.create_mask(src_tensor)
        
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    attentions = torch.zeros(max_len, 1, len(src_indexes)).to(device)
    
    for i in range(max_len):

        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)
                
        with torch.no_grad():
            output, hidden, attention,vis_attention = model.decoder(trg_tensor, hidden, encoder_outputs,vis_features, mask)

        attentions[i] = attention
            
        pred_token = output.argmax(1).item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attentions[:len(trg_tokens)-1]

In [86]:
'''def display_attention(sentence, translation, attention, n_heads = 8, n_rows = 4, n_cols = 2):
    
    assert n_rows * n_cols == n_heads
    
    fig = plt.figure(figsize=(15,25))
    
    for i in range(n_heads):
        
        ax = fig.add_subplot(n_rows, n_cols, i+1)
        
        _attention = attention.squeeze(0)[i].cpu().detach().numpy()

        cax = ax.matshow(_attention, cmap='bone')

        ax.tick_params(labelsize=12)
        ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                           rotation=45)
        ax.set_yticklabels(['']+translation)

        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()
'''

"def display_attention(sentence, translation, attention, n_heads = 8, n_rows = 4, n_cols = 2):\n    \n    assert n_rows * n_cols == n_heads\n    \n    fig = plt.figure(figsize=(15,25))\n    \n    for i in range(n_heads):\n        \n        ax = fig.add_subplot(n_rows, n_cols, i+1)\n        \n        _attention = attention.squeeze(0)[i].cpu().detach().numpy()\n\n        cax = ax.matshow(_attention, cmap='bone')\n\n        ax.tick_params(labelsize=12)\n        ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], \n                           rotation=45)\n        ax.set_yticklabels(['']+translation)\n\n        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))\n        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))\n\n    plt.show()\n    plt.close()\n"

In [87]:
'''text_idx = 26                #renamed from 'example_idx'
image_idx="9"

path=r"visual_features_resnet.csv"

src = vars(train_data.examples[text_idx])['src']
trg = vars(train_data.examples[text_idx])['trg']
#img = vars(train_data.examples[text_idx])['img_path']

translation, attention = translate_sentence(src, SRC, TRG, image_idx,path, model, device)

#print(f'src = {src}\n')
print(f'Ground-truth Comment')
print( '--------------------')
print(f'trg = {trg}')

print(f'\n\nPredicted Feedback (Feedback)')
print( '-----------------------------')
    
print(f'{translation}')
'''

'text_idx = 26                #renamed from \'example_idx\'\nimage_idx="9"\n\npath=r"visual_features_resnet.csv"\n\nsrc = vars(train_data.examples[text_idx])[\'src\']\ntrg = vars(train_data.examples[text_idx])[\'trg\']\n#img = vars(train_data.examples[text_idx])[\'img_path\']\n\ntranslation, attention = translate_sentence(src, SRC, TRG, image_idx,path, model, device)\n\n#print(f\'src = {src}\n\')\nprint(f\'Ground-truth Comment\')\nprint( \'--------------------\')\nprint(f\'trg = {trg}\')\n\nprint(f\'\n\nPredicted Feedback (Feedback)\')\nprint( \'-----------------------------\')\n    \nprint(f\'{translation}\')\n'

### Spice & Meteor
#### (Hitting erorr in py3, need to run in py2...) tried switching the kernel to py2 and runnign here but T_EPOCHS is cleared off...
#### 1. Go to '/home/puneet/code/EvalMetrics_py2'
#### 2. Copy 'nEp_test_comments.csv' & 'nEp_test_feedbacks.csv'
#### 3. Rename as 'test_comments.csv' & 'test_feedbacks.csv'
#### 4. Run eval.py with 'py2' env
#### 5. Rename the generated 'scores.txt' file  

In [88]:
def quant_eval(model, optimizer, start_epoch, chpt_file):
    from pycocoevalcap.bleu.bleu import Bleu
    from pycocoevalcap.cider.cider import Cider
    from pycocoevalcap.meteor.meteor import Meteor
    from pycocoevalcap.rouge.rouge import Rouge
    from pycocoevalcap.spice.spice import Spice
    import os, json, csv
    
    model, optimizer, start_epoch = load_checkpt(model, optimizer, chpt_file)
    T_EPOCHS = start_epoch + EP_INT    
    

    #print('Download Stanford models... Run once!')
    os.system("sh get_stanford_models.sh")

    with open('temp/'+str(T_EPOCHS)+'Ep_test_comments.csv',"r") as f: 
            reader = csv.reader(f)
            gts = {rows[0]:rows[1:] for rows in reader}
            #print(mydict) #prints with single quotes
            #print (json.dumps(mydict)) #prints with double quotes

    with open('temp/'+str(T_EPOCHS)+'Ep_test_feedbacks.csv',"r") as g: 
            reader = csv.reader(g)
            res = {rows[0]:rows[1:] for rows in reader}
            #print(json.dumps(mydict))

    '''with open('temp/test_comments.json', 'r') as file:
        gts = json.load(file)
    with open('temp/test_feedbacks.json', 'r') as file:
        res = json.load(file)
    '''

    def bleu():
        scorer = Bleu(n=4)
        score, scores = scorer.compute_score(gts, res)
        return score


    def cider():
        scorer = Cider()
        (score, scores) = scorer.compute_score(gts, res)
        return score

    def rouge():
        scorer = Rouge()
        score, scores = scorer.compute_score(gts, res)
        return score

    #bgts = gts[0].encode(encoding='UTF-8')
    #bres = res[0].encode(encoding='UTF-8')

    def spice():
        scorer = Spice()
        #print(gts, res)
        score, scores = scorer.compute_score(gts, res)
        return score

    def meteor():
        scorer = Meteor()
        #print(gts, res)
        score, scores = scorer.compute_score(bgts, bres)
        return score    
    s_cider=cider()
    s_rouge=rouge()
    s_bleu=bleu()
    #s_spice=spice()#
    #s_meteor=meteor()#
    
    print('\n----------------------\nbleu = %s' %s_bleu )
    print('cider = %s' %s_cider )
    print('rouge = %s' %s_rouge )
    #print('spice = %s' %s_spice )
    #print('meteor = %s' %s_meteor )
    
    b=" ".join(str(x) for x in s_bleu)
    print('\n----------------------')
    f = open('scores.txt', 'w') 
    f.write("\ncider: %f" % s_cider)
    f.write("\nrouge: %f" % s_rouge)
    #f.write("\nspice: %f" % s_spice)
    #f.write("\nmeteor: %f" % s_meteor)
    f.write("\nbleu :")
    f.write(b)
    f.close()
    
    #print(str(T_EPOCHS))
    #Log with Tensorboard: Eval metrics
    log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/cider', str(s_cider))
    log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/rouge', str(s_rouge))
    #log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/spice', str(s_spice))
    #log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/meteor', str(s_meteor))
    log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/bleu-1', str(s_bleu[0]))
    log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/bleu-2', str(s_bleu[1]))
    log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/bleu-3', str(s_bleu[2]))
    log_writer.add_text(str(T_EPOCHS)+'Ep=>Metrics/bleu-4', str(s_bleu[3]))

In [89]:
def qual_eval(model, optimizer, start_epoch, chpt_file):
    model, optimizer, start_epoch = load_checkpt(model, optimizer, chpt_file)
    T_EPOCHS = start_epoch + EP_INT
    
    # Save the predicted Feedbacks in CSV file
    # Log with Tensorboard: Text, Comment, Image and Feedback
    test_pred=[]
    #test_df=pd.read_csv("temp/test_data.csv")  
    test_df=pd.read_csv("temp/test_data1.csv") 
    path=r"visual_features_resnet.csv"      
    length=test_df.shape[0]
    images = []

    #print(length)
    for i in range(length):
        src=test_df['src'][i]
        trg=test_df['trg'][i]
        img_path=test_df['img_path'][i]
        image_idx=test_df['img_id'][i]
        translation, attention = translate_sentence(src, SRC, TRG, image_idx, path, model, device)

        if not translation:
            translation="*empty*"

        #Untokenization    
        translation1=translation[0:(len(translation)-1)]    
        translation2 = TreebankWordDetokenizer().detokenize(translation1)
        test_pred.append(str(translation2))

        image = Image.open(img_path)
        image = ToTensor()(image)   
        #images.append(image)
        
        #print(str(T_EPOCHS))
        if (i%10==0): 
            #Log with Tensorboard: Text, Comment, Image and Feedback
            log_writer.add_text(str(T_EPOCHS)+'Ep=>Ground-truth Comment of Sample/'+str(i+1), str(trg))
            log_writer.add_text(str(T_EPOCHS)+'Ep=>News Text of Sample/'+str(i+1), str(src))#, i+1)
            log_writer.add_text(str(T_EPOCHS)+'Ep=>Predicted Feedback of Sample/'+str(i+1), str(translation))
            #log_writer.add_image('Image', image, i+1)
            #image_grid = torchvision.utils.make_grid(images)
            log_writer.add_image(str(T_EPOCHS)+'Ep:Image of Sample/'+str(i+1), image)        

    with open('temp/'+str(T_EPOCHS)+'Ep_test_results.csv', 'w'): 
        pass
    with open('temp/'+str(T_EPOCHS)+'Ep_test_comments.csv', 'w'): 
        pass
    with open('temp/'+str(T_EPOCHS)+'Ep_test_feedbacks.csv', 'w'): 
        pass

    test_df["pred"] = test_pred 
    test_df.to_csv('temp/'+str(T_EPOCHS)+'Ep_test_results.csv', index= False)
    test_df.to_csv('temp/'+str(T_EPOCHS)+'Ep_test_comments.csv', index= True, columns=["trg"]) #index -> "key": value -> ["trg/pred"]
    test_df.to_csv('temp/'+str(T_EPOCHS)+'Ep_test_feedbacks.csv', index= True, columns=["pred"])

    #Re-open and save with new column names
    df = pd.read_csv('temp/'+str(T_EPOCHS)+'Ep_test_comments.csv')
    df.columns = ['id', 'comment']
    df.to_csv('temp/'+str(T_EPOCHS)+'Ep_test_comments.csv', index= False)

    df = pd.read_csv('temp/'+str(T_EPOCHS)+'Ep_test_feedbacks.csv')
    df.columns = ['id', 'feedback']
    df.to_csv('temp/'+str(T_EPOCHS)+'Ep_test_feedbacks.csv', index= False)
    
    print('tensorboard --logdir "/home/puneet/code/Multimodal Feedback/TBlogs"\n---')

In [90]:
def interval_train(model, optimizer, start_epoch, chpt_file):
    model, optimizer, start_epoch = load_checkpt(model, optimizer, chpt_file)
    T_EPOCHS = start_epoch + EP_INT

    print('Already trained for',start_epoch, 'epochs. Training now for', EP_INT, 'more')

    best_valid_loss = float('inf')
    cur_best_train_loss = float('inf')

    #for epoch in range(EP_INT):
    for epoch in range(start_epoch, T_EPOCHS):    
        start_time = time.time()

        train_loss = train(model, train_iterator, optimizer, criterion, CLIP)#, log_writer_train)
        valid_loss = evaluate(model, valid_iterator, criterion)#, log_writer_val)

        #Log with Tensorboard: Loss & PPL for train & val 
        log_writer.add_scalar('Train/Loss',float(train_loss), epoch+1)
        log_writer.add_scalar('Train/PPL', float(math.exp(train_loss)), epoch+1)    
        log_writer.add_scalar('Val/Loss',float(valid_loss), epoch+1)
        log_writer.add_scalar('Val/PPL', float(math.exp(valid_loss)), epoch+1)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        #if valid_loss < best_valid_loss:
        #    best_valid_loss = valid_loss
        #if train_loss < cur_best_train_loss:
        #    cur_best_train_loss = train_loss

        state = {'epoch': T_EPOCHS, 'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict(), 'loss': train_loss}
        torch.save(state, chpt_file)

        print(f'\nEpoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

    torch.cuda.empty_cache()

In [91]:
chpt_file = '/home/puneet/code/Multimodal Feedback/checkpoints/baseline2.pt'
model, optimizer, start_epoch = load_checkpt(model, optimizer, chpt_file)
T_EPOCHS = start_epoch + EP_INT
    
for i in range(EPOCHS):
    if (i%EP_INT==0):
        interval_train(model, optimizer, start_epoch, chpt_file)
        print('\nEvaluation\n----------------------')
        qual_eval(model, optimizer, start_epoch, chpt_file)
        quant_eval(model, optimizer, start_epoch, chpt_file)

=> Checkpoint NOT found '/home/puneet/code/Multimodal Feedback/checkpoints/baseline2.pt'
=> Checkpoint NOT found '/home/puneet/code/Multimodal Feedback/checkpoints/baseline2.pt'
Already trained for 0 epochs. Training now for 1 more

Epoch: 01 | Time: 5m 54s
	Train Loss: 7.035 | Train PPL: 1135.835
	 Val. Loss: 6.966 |  Val. PPL: 1060.089

Evaluation
----------------------
=> loading checkpoint '/home/puneet/code/Multimodal Feedback/checkpoints/baseline2.pt'
=> loaded checkpoint '/home/puneet/code/Multimodal Feedback/checkpoints/baseline2.pt' (epoch 1)
tensorboard --logdir "/home/puneet/code/Multimodal Feedback/TBlogs"
---
=> loading checkpoint '/home/puneet/code/Multimodal Feedback/checkpoints/baseline2.pt'
=> loaded checkpoint '/home/puneet/code/Multimodal Feedback/checkpoints/baseline2.pt' (epoch 1)
{'testlen': 96, 'reflen': 1280, 'guess': [96, 0, 0, 0], 'correct': [38, 0, 0, 0]}
ratio: 0.0749999999999414

----------------------
bleu = [1.7426643764117067e-06, 2.7698584248886856e-09,

RuntimeError: CUDA out of memory. Tried to allocate 30.00 MiB (GPU 0; 7.80 GiB total capacity; 4.06 GiB already allocated; 61.12 MiB free; 4.42 GiB reserved in total by PyTorch)